# 1. as_retriever

- search_type 参数：
    - similarity: 默认值
    - mmr: Maximal Marginal Relevance, 最大边际相关性
    - similarity_score_threshold: 相似度得分阈值,取值\[0,1.0\],0代表不相似，1.0代表完全相同
- search_kwargs: 配合search_type设置，search_type的不同取值，search_kwargs有不同设置
    - similarity: {"k": 20}
    - mmr: {"k": 4, "fetch_k": 20, "lambda_mult": 0.5}
    - similarity_score_threshold: {'k': 20, 'score_threshold': 0.2}


In [21]:
# 1. 添加文档到chromadb当中
# 2. 根据问题进行检索并输出答案

import chromadb
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser

def get_metadata_str(metadata):
    if metadata is None:
        return ""
    _str = ""
    # (key, value)
    for key, value in metadata.items():
        _str += f"{key}:{value}\n"
    return _str

class RagLangChain():
    def __init__(self, 
                 collection_name: str = "rag-video-collection", 
                 host: str = "localhost",
                port: int = 8000):
        self.collection_name = collection_name
        self.host = host
        self.port = port

    def __get_vector_store(self):
        chromadb_client = chromadb.HttpClient(host=self.host, port=self.port)
        embedding_fn = OllamaEmbeddings(model="nomic-embed-text:latest")
        
        vector_store = Chroma(collection_name=self.collection_name,
                              client=chromadb_client, 
                              embedding_function=embedding_fn
                             )
        return vector_store

    def delete_collection(self):
        vector_store = self.__get_vector_store()
        vector_store.delete_collection()
        
    def add_file(self, file_path: str, metadata = None):
        loader = TextLoader(file_path)
        
        docs = loader.load()
        text_spliter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
        all_splits = text_spliter.split_documents(docs)

        if metadata is not None:
            for split_item in all_splits:
                split_metadata = split_item.metadata
                split_item.metadata = {**split_metadata, **metadata}
        
        vector_store = self.__get_vector_store()
        ids = vector_store.add_documents(documents=all_splits)
        return ids

    def __query_vector(self, info):
        vector_store = self.__get_vector_store()
        retriever = vector_store.as_retriever(
            search_type="mmr",
            search_kwargs={"k": 16, "fetch_k": 20, "lambda_mult": 0.1}
        )
        docs = retriever.invoke(info["query"])
        # (0, item0), (1, item1)
        for index, doc_item in enumerate(docs):
            print(f"文档片段{index}")
            print(doc_item)
            print("*" * 80)
        # docs_str = "\n\n".join(doc.page_content for doc in docs)
        docs_str = "\n\n".join(f"""
{get_metadata_str(doc.metadata)}
{doc.page_content}""" for doc in docs)
        # print(f"@@@@@{docs_str}")
        return docs_str
        
    def query(self, question: str):
        prompt = ChatPromptTemplate.from_template("""
你是一个问答机器人。你的任务是根据下述给定的已知信息回答用户问题。

已知信息:
{context}
用户问题：
{query}
如果已知信息不包含用户问题的答案，或者已知信息不足以回答用户的问题，请直接回复"我无法回答您的问题"。
请不要输出已知信息中不包含的信息或答案。
请用中文回答用户问题。
""")
        
        llm = ChatOllama(model="qwen2.5:latest")
        
        output_parser = StrOutputParser()
        # 返回值: {"query": "XXXX", "context": "XXXXX2"}
        chain = ( {"context": self.__query_vector, "query": lambda x: x["query"]} | prompt | llm | output_parser)
        
        result = chain.invoke({"query": question})
        return result


In [22]:
rag_chain = RagLangChain()

In [23]:
rag_chain.query(question="2024年中东非智能手机的出货量为多少？")

文档片段0
page_content='三、智能手机:2024 年、2025 年连续两年保持增长
预计 2025 年全球智能手机出货量将同比增长 2%
根据 IDC 数据，今年三季度全球智能手机出货量 3.15 亿部，环比增长 8%， 同比增长4%(图表 34)。虽然同比增速较一二季度下行，但是同比增速略 好于我们此前预测。这与我们与智能手机供应链沟通下来的情况类似，比半 年报看到的行业需求情况略好。' metadata={'source': '科技行业 2025 年展望.txt', '作者': '沈岱,马智焱,黄佳琦', '发表时间': '2024 年 12 月 13 日', '文章标题': '科技行业2025年展望'}
********************************************************************************
文档片段1
page_content='进一步看，明年高端智能手机出货量有望继续跑赢整体大盘。其一，从供应 端来看，明年会更多端侧AI 手机上市，从给高端智能手机带来新的产品。 其次，从需求角度看，端侧AI 手机更加容易刺激高端用户的换机需求。配 合长换机期下的高端需求增量，高端智能手机需求也有望继续提升。' metadata={'source': '科技行业 2025 年展望.txt', '作者': '沈岱,马智焱,黄佳琦', '发表时间': '2024 年 12 月 13 日', '文章标题': '科技行业2025年展望'}
********************************************************************************
文档片段2
page_content='高端机型相对优异的表现与我们在安卓品牌端以及供应链端看到的情况类 似。虽然全球智能手机换机周期仍然处于偏高位置，但是在较长换机周期的 情况下，用户愿意提升预算购买相对高端的机型。而千元以下手机占比提升， 主要来自于中东非等发展地区手机出货量更加强劲的表现。这可能是美国进 入降息周期带动这些地区手机需求的释放。
其中，小米的高端机型表现比较典型。今年三季度，小米 3,000 元人民币以 上的智能手机出货量占到小米总出货量的6.3%，同比增加 1.5 个百分点，环 比

'2024年中东非智能手机的出货量为4,199万部。'